In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import Column
from pyspark.sql.functions import upper,split,trim,ascii,substring,substring_index
from pyspark.sql.types import DecimalType,IntegerType

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
degree='°'

22/08/15 13:30:18 WARN Utils: Your hostname, debianwork resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/08/15 13:30:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/15 13:30:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_temps = spark.read.option("header",True).csv("../../data/global_sea_temp/SeaTemperatures.csv")
df_temps.show(10)

+---------+---------------+---------+--------+
|     City|       CurrTemp|WindSpeed|Humidity|
+---------+---------------+---------+--------+
|  Algiers|23.6°C / 74.5°F|   14 mph|     65%|
|   Annaba|24.9°C / 76.8°F|    3 mph|     88%|
|      Bab|23.5°C / 74.3°F|    5 mph|     52%|
|   Bejaïa|24.3°C / 75.7°F|    5 mph|     78%|
|     Beni|21.8°C / 71.2°F|    4 mph|     75%|
|    Bordj|23.5°C / 74.3°F|    8 mph|     88%|
|Boumerdas|23.4°C / 74.2°F|   13 mph|     55%|
|   Dellys|24.5°C / 76.1°F|    8 mph|     59%|
|    Jijel|24.3°C / 75.7°F|    2 mph|     83%|
|     Mers|21.8°C / 71.2°F|    6 mph|     88%|
+---------+---------------+---------+--------+
only showing top 10 rows



In [9]:
df_temps2 = df_temps.select(
    df_temps.City,
    substring_index(
        split(df_temps.CurrTemp,'/').getItem(0),degree,1
    ).cast(DecimalType(4,1)).alias('Temp_C'),
    substring_index(
        split(df_temps.CurrTemp,'/').getItem(1),degree,1
    ).cast(DecimalType(4,1)).alias('Temp_F'),
    substring_index(df_temps.WindSpeed,' mph',1).cast('integer').alias('WindSpeed_mph'),
    (substring_index(df_temps.Humidity,'%',1).cast('integer') / 100).alias('Humidity')
)
df_temps2.show(10)    

+---------+------+------+-------------+--------+
|     City|Temp_C|Temp_F|WindSpeed_mph|Humidity|
+---------+------+------+-------------+--------+
|  Algiers|  23.6|  74.5|           14|    0.65|
|   Annaba|  24.9|  76.8|            3|    0.88|
|      Bab|  23.5|  74.3|            5|    0.52|
|   Bejaïa|  24.3|  75.7|            5|    0.78|
|     Beni|  21.8|  71.2|            4|    0.75|
|    Bordj|  23.5|  74.3|            8|    0.88|
|Boumerdas|  23.4|  74.2|           13|    0.55|
|   Dellys|  24.5|  76.1|            8|    0.59|
|    Jijel|  24.3|  75.7|            2|    0.83|
|     Mers|  21.8|  71.2|            6|    0.88|
+---------+------+------+-------------+--------+
only showing top 10 rows

